## Autograd

In [1]:
from typing import Union, List

import numpy as np
from numpy import ndarray as array

In [2]:
a = 3
a.__add__(4)

7

In [3]:
np.array(3)

array(3)

In [4]:
c = array(3)
c

array([0.75, 0.75, 0.  ])

In [5]:
a = array([3,3])
print("Addition using '__add__':", a.__add__(4))
print("Addition using '+':", a + 4)

Addition using '__add__': [[4. 4. 4.]
 [4. 4. 4.]
 [4. 4. 4.]]
Addition using '+': [[4. 4. 4.]
 [4. 4. 4.]
 [4. 4. 4.]]


In [8]:
Numberable = Union[float, int]

def ensure_number(num: Numberable):
    if isinstance(num, NumberWithGrad):
        return num
    else:
        return NumberWithGrad(num)        

class NumberWithGrad(object):
    
    def __init__(self, 
                 num: Numberable,
                 depends_on: List[Numberable] = None,
                 creation_op: str = ''):
        self.num = num
        self.grad = None
        self.depends_on = depends_on or []
        self.creation_op = creation_op

    def __add__(self, 
                other: Numberable):
        return NumberWithGrad(self.num + ensure_number(other).num,
                              depends_on = [self, ensure_number(other)],
                              creation_op = 'add')
    
    def __mul__(self,
                other: Numberable = None):

        return NumberWithGrad(self.num * ensure_number(other).num,
                              depends_on = [self, ensure_number(other)],
                              creation_op = 'mul')
    
    def backward(self, backward_grad: Numberable = None):
        if backward_grad is None: # first time calling backward
            self.grad = 1
        else: 
            # These lines allow gradients to accumulate.
            # If the gradient doesn't exist yet, simply set it equal
            # to backward_grad
            if self.grad is None:
                self.grad = backward_grad
            # Otherwise, simply add backward_grad to the existing gradient
            else:
                self.grad += backward_grad
        
        if self.creation_op == "add":
            # Simply send backward self.grad, since increasing either of these 
            # elements will increase the output by that same amount
            self.depends_on[0].backward(self.grad)
            self.depends_on[1].backward(self.grad)    

        if self.creation_op == "mul":

            # Calculate the derivative with respect to the first element
            new = self.depends_on[1] * self.grad
            # Send backward the derivative with respect to that element
            self.depends_on[0].backward(new.num)

            # Calculate the derivative with respect to the second element
            new = self.depends_on[0] * self.grad
            # Send backward the derivative with respect to that element
            self.depends_on[1].backward(new.num)

In [9]:
a = NumberWithGrad(3)
b = a * 4
c = b + 3
c.backward()
print(a.grad)
print(b.grad)

1


In [7]:
a = NumberWithGrad(3)

In [8]:
b = a * 4
c = b + 3
d = (a + 2)
e = c * d 
e.backward()

In [9]:
a.grad

35

## PyTorch

In [10]:
import torch
from torch import Tensor

In [11]:
torch.manual_seed(20190324)
array_data = np.full((2,2), 3.0)
print(array_data)
a = torch.tensor(array_data, requires_grad=True)
print(a)

[[3. 3.]
 [3. 3.]]
tensor([[3., 3.],
        [3., 3.]], dtype=torch.float64, requires_grad=True)


In [12]:
b = a * 4
c = b + 3
d = (a + 2)
e = c * d
e_sum = e.sum()
e_sum.backward()

In [13]:
a.grad

tensor([[35., 35.],
        [35., 35.]], dtype=torch.float64)